In [1]:
import os
import requests
from bs4 import BeautifulSoup

base_url = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/{year}'

years = range(2000, 2024)
os.makedirs('data', exist_ok=True)
for year in years:
    os.makedirs(f'data/{year}', exist_ok=True)
    url = base_url.format(year)
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    table = soup.find('table')
    anchors = table.find_all('a')
    anchors = [a for a in anchors if 'csv' in a.text]
    for anchor in anchors:
        file = anchor.text
        file_url = f'{url}/{file}'
        res = requests.get(file_url)
        csv = res.text
        with open(f'data/{year}/{file}', 'w') as f:
            f.write(csv)

In [38]:
# %pip install apache-airflow

/var/folders/6s/8vqr9nzs6bs835b3cl0fxc_w0000gn/T/ipykernel_21397/3869197976.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [43]:
import os
import requests
from bs4 import BeautifulSoup
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

def scrape_data(year):
    base_url = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/{year}'
    os.makedirs(f'data/{year}', exist_ok=True)
    url = base_url.format(year)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find('table')
    anchors = table.find_all('a')
    anchors = [a for a in anchors if 'csv' in a.text]
    for anchor in anchors:
        file = anchor.text
        file_url = f'{url}/{file}'
        res = requests.get(file_url)
        csv = res.text
        with open(f'data/{year}/{file}', 'w') as f:
            f.write(csv)

/var/folders/6s/8vqr9nzs6bs835b3cl0fxc_w0000gn/T/ipykernel_21397/4036876447.py:5 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [41]:
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1),
    'retries': 1,
}

dag = DAG(
    'noaa_data_scraping',
    default_args=default_args,
    schedule_interval=None,  # Set the schedule_interval as needed
)

for year in range(2000, 2024):
    task_id = f'scrape_data_{year}'
    scrape_task = PythonOperator(
        task_id=task_id,
        python_callable=scrape_data,
        op_args=[year],
        dag=dag,
    )

/var/folders/6s/8vqr9nzs6bs835b3cl0fxc_w0000gn/T/ipykernel_21397/2113139553.py:7 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [42]:
docker build -t my_airflow_image ./
docker compose up
docker exec -it 8ea0cd0a78a2 airflow dags trigger -r 123345 task_1_dag



In [2]:
import os
import requests
from bs4 import BeautifulSoup
# from airflow import DAG
# from airflow.operators.python_operator import PythonOperator
from datetime import datetime

def scrape_data(ds, year, **kwargs):
    base_url = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/{year}'
    os.makedirs(f'data/{year}', exist_ok=True)
    url = base_url.format(year=year)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find('table')
    anchors = table.find_all('a')
    anchors = [a for a in anchors if 'csv' in a.text]
    for anchor in anchors:
        file = anchor.text
        file_url = f'{url}/{file}'
        res = requests.get(file_url)
        csv = res.text
        with open(f'data/{year}/{file}', 'w') as f:
            f.write(csv)


In [3]:
scrape_data(None,2020)

KeyboardInterrupt: 